# Northward Winds of Chile
Kylene Cooley 

Oregon State University, CEOAS 

8/21/20: Adaption from the MatLab script I used in my OC 682 Term Project to Python

In [1]:
# supressing some warnings
import warnings 
warnings.filterwarnings("ignore") 

# import libraries
import datetime as dt
import xarray as xr
import fsspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html") 

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Goal: open netCDF file into nice dataset using xarray
